In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split #used to split your dataset into training and testing subsets.
from sklearn.preprocessing import StandardScaler, OneHotEncoder #standardizing features (variables) by scaling them 
#convert categorical text or numerical values into a format that machine learning algorithms can use
from sklearn.compose import ColumnTransformer #allows you to apply different preprocessing steps to different columns
from sklearn.pipeline import Pipeline #chains together multiple processing steps
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime
import re

In [35]:
IT_DATA = pd.read_csv('Employee_Profile_IT.csv')

In [36]:
IT_DATA.columns = IT_DATA.columns.str.replace(r'[^\w\s]', '', regex=True) #clean column names
IT_DATA.rename(columns={'Bonus ': 'Cur_Bonus'}, inplace=True) # Cur_Bonus = Current_Bonus_Ratio
IT_DATA.rename(columns={'Exit Date': 'Init_Exit'}, inplace=True) 
IT_DATA['Annual Salary'] = IT_DATA['Annual Salary'].replace({r'[$,]': ''}, regex=True).astype(float) #remove dollar sign
IT_DATA['Cur_Bonus'] = IT_DATA['Cur_Bonus'].astype(float)
IT_DATA.drop('Init_Exit', axis=1, inplace=True) #drop exit column
IT_DATA = IT_DATA[~IT_DATA['EEID'].str.contains('E100', na=False)].reset_index(drop=True) #filter employees

In [37]:
num_feats = ['Age', 'Annual Salary', 'Cur_Bonus', 'EmploymentRating', 'DaysOfAbsence', 'CertificationsEarned']
for feat in num_feats:
    if IT_DATA[feat].isnull().any():
        med_val = IT_DATA[feat].median() #because of outlires
        IT_DATA[feat].fillna(med_val, inplace=True)
for col in IT_DATA.columns:
    if IT_DATA[col].dtype == 'object' and IT_DATA[col].isnull().any():
        mode_val = IT_DATA[col].mode()[0]
        IT_DATA[col].fillna(mode_val, inplace=True)

print("Data Cleanup complete. IT dataset ready.")
print("Total IT records:", len(IT_DATA))

Data Cleanup complete. IT dataset ready.
Total IT records: 215


C:\Users\daree\AppData\Local\Temp\ipykernel_21016\3908407854.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  IT_DATA[feat].fillna(med_val, inplace=True)
C:\Users\daree\AppData\Local\Temp\ipykernel_21016\3908407854.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [38]:
Exit_Data = pd.read_csv('Employee_Exit.csv') 

In [39]:
Exit_Data.rename(columns={'Bonus %': 'Bonus_Train'}, inplace=True) 

In [40]:
Exit_Data['Annual Salary'] = Exit_Data['Annual Salary'].replace({r'[$,]': ''}, regex=True).astype(float)
Exit_Data['Bonus_Train'] = Exit_Data['Bonus_Train'].replace({r'%': ''}, regex=True).astype(float) / 100

In [41]:
DT_FMT = '%m/%d/%Y'
Exit_Data['Hire Date'] = pd.to_datetime(Exit_Data['Hire Date'], format=DT_FMT, errors='coerce')
Exit_Data['Exit Date'] = pd.to_datetime(Exit_Data['Exit Date'], format=DT_FMT, errors='coerce')
Exit_Data['Tenure_Days'] = (Exit_Data['Exit Date'] - Exit_Data['Hire Date']).dt.days

In [42]:
Exit_Data.dropna(subset=['Tenure_Days', 'Annual Salary', 'Bonus_Train'], inplace=True)

In [43]:
X_COLS = ['Age', 'Annual Salary', 'Bonus_Train', 'Country', 'Gender', 
          'Job Title', 'Department', 'Business Unit']
Y_TARG = 'Tenure_Days'
X_train = Exit_Data[X_COLS]
Y_train = Exit_Data[Y_TARG]

In [44]:
# apply different preprocessing steps to different subsets of columns
preproc = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), ['Age', 'Annual Salary', 'Bonus_Train']), #scales the numerical features so that they have a mean of 0 and a standard deviation of 1.
        ('encode', OneHotEncoder(handle_unknown='ignore'), ['Country', 'Gender', 'Job Title', 'Department', 'Business Unit']) #converts categorical text data into a numerical format that machine learning models can understand.
    ]
)

In [45]:
X_T, X_V, Y_T, Y_V = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [46]:
#chain together multiple data processing and modeling steps so that they can be treated as a single unit.
model_lin = Pipeline(steps=[
    ('preproc', preproc),
    ('reg', LinearRegression()) ])
#model is trained using the preprocessed data outputted by the previous step (preproc) and the target variable ($Y$)

In [47]:
model_lin.fit(X_T, Y_T)
print("Model trained: Linear Regression for Tenure.")

Model trained: Linear Regression for Tenure.


In [48]:
preds_V = model_lin.predict(X_V)
preds_V[preds_V < 0] = 0

In [49]:
err_mae = mean_absolute_error(Y_V, preds_V) #measures the average magnitude of the errors in a set of predictions.
err_mse = mean_squared_error(Y_V, preds_V) #measures the average of the squares of the errors
err_rmse = np.sqrt(err_mse)

In [50]:
print(f"\nModel Performance:")
print(f"MAE: {err_mae:.2f} days")
print(f"RMSE: {err_rmse:.2f} days")


Model Performance:
MAE: 866.06 days
RMSE: 997.81 days


In [51]:
IT_INPUT = IT_DATA.copy()

In [52]:
IT_INPUT.rename(columns={'Cur_Bonus': 'Bonus_Train'}, inplace=True) 

In [53]:
pred_tenure = model_lin.predict(IT_INPUT[X_COLS])
pred_tenure[pred_tenure < 0] = 0

In [54]:
IT_INPUT['Hire Date'] = pd.to_datetime(IT_INPUT['Hire Date'], format=DT_FMT, errors='coerce')
tenure_td = pd.to_timedelta(pred_tenure, unit='D')
pred_exit_dt = IT_INPUT['Hire Date'] + tenure_td

In [55]:
IT_DATA['Predicted_ExitDate'] = pred_exit_dt.dt.strftime(DT_FMT)

In [56]:
print(f"\nPrediction Complete. Predicted Exit Dates appended to IT_DATA.")
print("--- Sample Predictions ---")
print(IT_DATA[['Full Name', 'Hire Date', 'Predicted_ExitDate']].head().to_string())


Prediction Complete. Predicted Exit Dates appended to IT_DATA.
--- Sample Predictions ---
          Full Name  Hire Date Predicted_ExitDate
0  Lillian Gonzales  3/13/2009         05/10/2014
1  Scarlett Jenkins  11/9/2011         06/21/2016
2  Brooklyn Salazar   3/1/2011         03/10/2016
3       Riley Rojas  1/21/2021         01/16/2026
4    Isabella Scott  4/26/2016         06/01/2021


In [57]:
output_filename = 'DataPredictedExit.csv'
IT_DATA.to_csv(output_filename, index=False)

In [58]:
# Hyperparameter Tuning
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

# Define the parameter grid for alpha
# Testing a wider, more aggressive range of alpha values
param_grid = {
    'reg__alpha': [0.1, 1.0, 10.0, 100.0, 500.0, 1000.0]
}

# Create a fresh Ridge pipeline
ridge_pipeline = Pipeline(steps=[
    ('preproc', preproc),
    ('reg', Ridge(random_state=42)) 
])

# Setup GridSearchCV using the training data (X_train, Y_train_enh)
# We use 'neg_mean_squared_error' as the scoring metric for optimization
grid_search = GridSearchCV(
    ridge_pipeline, 
    param_grid, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    verbose=1
)

# Fit the grid search model
grid_search.fit(X_train, Y_train_enh)

# Get the best model and parameters
best_ridge_model = grid_search.best_estimator_
best_alpha = grid_search.best_params_['reg__alpha']

print(f"Optimal Alpha found: {best_alpha}")

# Evaluate the best model
preds_V_tuned = best_ridge_model.predict(X_V)
preds_V_tuned[preds_V_tuned < 0] = 0

# Convert back to days for consistency
preds_V_days_tuned = preds_V_tuned * 365.25

err_mae_tuned = mean_absolute_error(Y_V_days_enh, preds_V_days_tuned)
err_rmse_tuned = np.sqrt(mean_squared_error(Y_V_days_enh, preds_V_days_tuned))


print(f"\n--- Ridge Model Performance (Tuned $\\alpha$={best_alpha}) ---")
print(f"Enhanced MAE (Tuned): {err_mae_tuned:.2f} days")
print(f"Enhanced RMSE (Tuned): {err_rmse_tuned:.2f} days")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Optimal Alpha found: 1000.0

--- Ridge Model Performance (Tuned $\alpha$=1000.0) ---
Enhanced MAE (Tuned): 864.73 days
Enhanced RMSE (Tuned): 990.29 days


In [61]:
pip install xgboost

   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/72.0 MB 628.5 kB/s eta 0:01:54
   ---------------------------------------- 0.5/72.0 MB 628.5 kB/s eta 0:01:54
   ---------------------------------------- 0.8/72.0 MB 680.0 kB/s eta 0:01:45
   ---------------------------------------- 0.8/72.0 MB 680.0 kB/s eta 0:01:45
    --------------------------------------- 1.0/72.0 MB 699.9 kB/s eta 0:01:42
    --------------------------------------- 1.0/72.0 MB 699.9 kB/s eta 0:01:42
    --------------------------------------- 1.0/72.0 MB 699.9 kB/s eta 0:01:42
    --------------------------------------- 1.3/72.0 MB 554.4 kB/s eta 0:02:08
    --------------

In [64]:
from xgboost import XGBRegressor 
from sklearn.model_selection import train_test_split, RandomizedSearchCV
Exit_Data['Tenure_Years'] = Exit_Data['Tenure_Days'] / 365.25
X_COLS = ['Age', 'Annual Salary', 'Bonus_Train', 'Country', 'Gender', 
          'Job Title', 'Department', 'Business Unit']
Y_TARG_ENHANCED = 'Tenure_Years'
X_train = Exit_Data[X_COLS]
Y_train_enh = Exit_Data[Y_TARG_ENHANCED]
X_T_enh, X_V_enh, Y_T_enh, Y_V_enh = train_test_split(
    X_train, Y_train_enh, test_size=0.2, random_state=42
)
preproc_xg = ColumnTransformer(
    transformers=[
        # Numerical features passed through without scaling
        ('no_scale', 'passthrough', ['Age', 'Annual Salary', 'Bonus_Train']), 
        # Categorical features encoded
        ('encode', OneHotEncoder(handle_unknown='ignore'), ['Country', 'Gender', 'Job Title', 'Department', 'Business Unit'])
    ]
)
model_xgb_pipeline = Pipeline(steps=[
    ('preproc', preproc_xg), 
    ('reg', XGBRegressor(
        objective='reg:squarederror', 
        random_state=42, 
        n_jobs=-1
    ))
])
param_dist = {
    'reg__n_estimators': [100, 300, 500],
    'reg__max_depth': [3, 5, 7, 10],  # Prevents overfitting
    'reg__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'reg__subsample': [0.6, 0.8, 1.0],
    'reg__colsample_bytree': [0.6, 0.8, 1.0]
}
random_search = RandomizedSearchCV(
    model_xgb_pipeline, 
    param_distributions=param_dist, 
    n_iter=50, # Test 50 random combinations (can be increased for better search)
    scoring='neg_mean_squared_error', 
    cv=5, # 5-fold cross-validation
    verbose=1, 
    random_state=42,
    n_jobs=-1
)
random_search.fit(X_train, Y_train_enh)
best_xgb_model = random_search.best_estimator_
print(f"Optimal XGBoost Parameters found: {random_search.best_params_}")
preds_V_tuned = best_xgb_model.predict(X_V_enh)
preds_V_tuned[preds_V_tuned < 0] = 0
preds_V_days_tuned = preds_V_tuned * 365.25
Y_V_days_enh = Y_V_enh * 365.25
err_mae_tuned = mean_absolute_error(Y_V_days_enh, preds_V_days_tuned)
err_rmse_tuned = np.sqrt(mean_squared_error(Y_V_days_enh, preds_V_days_tuned))
print(f"\n--- Final Model Performance (Tuned XGBoost) ---")
# Replace 'Original' values with your own lowest-recorded MAE/RMSE for comparison
print(f"Original Linear Regression RMSE: [Insert Your Lowest RMSE Here] days") 
print(f"Final Tuned XGBoost MAE: {err_mae_tuned:.2f} days")
print(f"Final Tuned XGBoost RMSE: {err_rmse_tuned:.2f} days")

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Optimal XGBoost Parameters found: {'reg__subsample': 0.6, 'reg__n_estimators': 100, 'reg__max_depth': 5, 'reg__learning_rate': 0.01, 'reg__colsample_bytree': 0.6}

--- Final Model Performance (Tuned XGBoost) ---
Original Linear Regression RMSE: [Insert Your Lowest RMSE Here] days
Final Tuned XGBoost MAE: 831.30 days
Final Tuned XGBoost RMSE: 953.04 days


In [65]:
pred_tenure_final = best_xgb_model.predict(IT_INPUT[X_COLS])
pred_tenure_final[pred_tenure_final < 0] = 0
# Convert predicted tenure from years back to days for datetime calculation
pred_tenure_days_final = pred_tenure_final * 365.25
# Calculate predicted exit date
IT_INPUT_final = IT_DATA.copy()
# Restore Hire Date and DT_FMT (assuming it was "%m/%d/%Y")
DT_FMT = '%m/%d/%Y'
IT_INPUT_final['Hire Date'] = pd.to_datetime(IT_INPUT_final['Hire Date'], format=DT_FMT, errors='coerce')

tenure_td_final = pd.to_timedelta(pred_tenure_days_final, unit='D')
pred_exit_dt_final = IT_INPUT_final['Hire Date'] + tenure_td_final

# Append the final predicted exit date to the IT_DATA DataFrame
IT_DATA['Predicted_ExitDate_Final'] = pred_exit_dt_final.dt.strftime(DT_FMT)

print(f"\nPrediction Complete with Final Tuned XGBoost Model. New column added to IT_DATA.")
print("--- Final Sample Predictions ---")
print(IT_DATA[['Full Name', 'Hire Date', 'Predicted_ExitDate_Final']].head().to_string())

# Save the final data
output_filename_final = 'DataPredictedExit_enhanced.csv'
IT_DATA.to_csv(output_filename_final, index=False)

print(f"\nSaved file: {output_filename_final}")


Prediction Complete with Final Tuned XGBoost Model. New column added to IT_DATA.
--- Final Sample Predictions ---
          Full Name  Hire Date Predicted_ExitDate_Final
0  Lillian Gonzales  3/13/2009               08/09/2014
1  Scarlett Jenkins  11/9/2011               11/10/2016
2  Brooklyn Salazar   3/1/2011               04/29/2016
3       Riley Rojas  1/21/2021               02/09/2026
4    Isabella Scott  4/26/2016               04/07/2021

Saved file: DataPredictedExit_enhanced.csv
